In [ ]:
"""
    Requirements
Input the outputs from 2024 Week 1 challenge
Input the new targets Excel sheet (Q1 - 4) 
Correct the Classes being incorrect as per last week
Economy to First
First Class to Economy
Business Class to Premium
Premium Economy to Business
Find the First Letter from each word in the Class to help with joining the Targets data to Sales data
Change the date to a month number 
Total up the sales at the level of:
Class
Month
Join the Targets data on to the Sales data (note - you should have 48 rows of data after the join)
Calculate the difference between the Sales and Target values per Class and Month
Output the data
"""

import pandas as pd

flow_card_holders_df = pd.read_csv('../../../prepped_data/2024/week_1/flow_card_holders.csv')
non_flow_card_holders_df = pd.read_csv('../../../prepped_data/2024/week_1/non_flow_card_holders.csv')

# Concatenate the dataframes together
unioned_df = pd.concat([flow_card_holders_df, non_flow_card_holders_df])
unioned_df.to_csv('../../../prepped_data/2024/week_3/week_3_input.csv', index=False)

# Correct the Classes being incorrect as per last week
unioned_df['Class'] = unioned_df['Class'].replace({'Economy': 'First Class', 'First Class': 'Economy', 'Business Class': 'Premium Economy', 'Premium Economy': 'Business Class'})

# Convert Class to a string
unioned_df['Class'] = unioned_df['Class'].astype(str)
# Find the First Letter from each word in the Class to help with joining the Targets data to Sales data
unioned_df['Class'] = unioned_df['Class'].apply(lambda x: ''.join(word[0] for word in x.split()))

# Change the date to a month number
unioned_df['Date'] = pd.to_datetime(unioned_df['Date'], dayfirst=True, format='%d/%m/%Y').dt.month

# Total up the sales at the level of Class and Date
sales_df = pd.pivot_table(unioned_df, index=['Class', 'Date'], values=['Price'], aggfunc='sum').reset_index()

targets_df = pd.read_csv('../../../raw_data/2024/week_3/PD 2024 Wk 3 Input Q1.csv')
targets_df['Date'] = targets_df['Month']
targets_df.drop('Month', axis=1, inplace=True)

sales_df = pd.merge(sales_df, targets_df, on=['Class', 'Date'], how='left')

# Rename Price to Sales
sales_df = sales_df.rename(columns={'Price': 'Sales'})
# Calculate the difference between the Sales and Target values per Class and Date
sales_df['Difference to Target'] = sales_df['Sales'] - sales_df['Target']


# Round sales to 2 decimal places   
sales_df.round({'Sales': 2})
# Round difference to target to 2 decimal places
sales_df = sales_df.round({'Difference to Target': 2})

# Reorganize columns to match the solution
sales_df = sales_df[['Class', 'Sales', 'Target', 'Difference to Target', 'Date']]


# Read the solution file
# Note the posted solution is not correct. The corrected solution is here: https://community.tableau.com/s/contentdocument/069cw000003hY3NAAU
solution_df = pd.read_csv('../../../solutions/2024/week_3/PD2024W3_Performance_vs_Targets.csv')
solution_df['Date'] = solution_df['Month']

# Round difference to target to 2 decimal places
solution_df.round({'Difference to Target': 2})

solution_df.drop('Month', axis=1, inplace=True)

# Sort rows by date & class to compare dfs
solution_df = solution_df.sort_values(['Date', 'Class'])
solution_df.reset_index(drop=True, inplace=True)

# Sort the rows by 'Date' and 'Class'

sales_df = sales_df.sort_values(['Date', 'Class',])
sales_df.reset_index(drop=True, inplace=True)


# Convert date to int
sales_df['Date'] = sales_df['Date'].astype(int)
# Compare the dataframes to the provided solution
print(f'The sales dataframe is identical to the provided solution: {sales_df.equals(solution_df)}')
sales_df.to_csv('../../../prepped_data/2024/week_3/week_3_output.csv', index=False)

The sales dataframe is identical to the provided solution: True
